<a href="https://colab.research.google.com/github/ameyas1/Shakespeare_sonnet_generator/blob/master/Shakespeare_75.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tf-nightly-gpu
import tensorflow as tf

    100% |████████████████████████████████| 367.1MB 55kB/s 
    100% |████████████████████████████████| 409kB 12.2MB/s 
    100% |████████████████████████████████| 61kB 24.2MB/s 
    100% |████████████████████████████████| 3.0MB 6.4MB/s 


In [0]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import CuDNNLSTM
from keras.utils import np_utils

In [0]:
text=(open('ss.txt').read())
text=text.lower()

In [0]:
characters=sorted(list(set(text)))

In [0]:
n_to_char={n:char for n, char in enumerate(characters)}

In [0]:
char_to_n={char:n for n, char in enumerate(characters)}

In [0]:
x=[]
y=[]
length=len(text)
seq_length=100

In [0]:
for i in range(0,length-seq_length):
  sequence=text[i:i+seq_length]
  label=text[i+seq_length]
  x.append([char_to_n[char] for char in sequence])
  y.append(char_to_n[label])
  

In [0]:
x_modified=np.reshape(x,(len(x),seq_length,1))
x_modified=x_modified/float(len(characters))
y_modified=np_utils.to_categorical(y)

**Model**

In [0]:
model=Sequential()

In [0]:
model.add(CuDNNLSTM(600,input_shape=(x_modified.shape[1],x_modified.shape[2]),return_sequences=True))
model.add(Dropout(0.2))

In [0]:
model.add(CuDNNLSTM(600,return_sequences=True))
model.add(Dropout(0.2))

In [0]:
model.add(CuDNNLSTM(600))

In [0]:
model.add(Dropout(0.2))

In [0]:
model.add(Dense(y_modified.shape[1],activation='softmax'))

In [0]:
model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

In [78]:
x_modified.shape

(98863, 100, 1)

In [79]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_6 (CuDNNLSTM)     (None, 100, 600)          1447200   
_________________________________________________________________
dropout_6 (Dropout)          (None, 100, 600)          0         
_________________________________________________________________
cu_dnnlstm_7 (CuDNNLSTM)     (None, 100, 600)          2884800   
_________________________________________________________________
dropout_7 (Dropout)          (None, 100, 600)          0         
_________________________________________________________________
cu_dnnlstm_8 (CuDNNLSTM)     (None, 600)               2884800   
_________________________________________________________________
dropout_8 (Dropout)          (None, 600)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 38)               

In [80]:
model.fit(x_modified,y_modified,epochs=20,batch_size=50)

Epoch 1/20
98863/98863 [==============================] - 444s 4ms/sample - loss: 2.7154 - acc: 0.2500
Epoch 2/20
98863/98863 [==============================] - 445s 5ms/sample - loss: 2.2931 - acc: 0.3406
Epoch 3/20
98863/98863 [==============================] - 445s 5ms/sample - loss: 2.0825 - acc: 0.3927
Epoch 4/20
98863/98863 [==============================] - 445s 5ms/sample - loss: 1.9351 - acc: 0.4298
Epoch 5/20
98863/98863 [==============================] - 445s 5ms/sample - loss: 1.8210 - acc: 0.4570
Epoch 6/20
98863/98863 [==============================] - 445s 5ms/sample - loss: 1.7210 - acc: 0.4839
Epoch 7/20
98863/98863 [==============================] - 443s 4ms/sample - loss: 1.6368 - acc: 0.5022
Epoch 8/20
98863/98863 [==============================] - 445s 4ms/sample - loss: 1.5561 - acc: 0.5225
Epoch 9/20
98863/98863 [==============================] - 445s 4ms/sample - loss: 1.4770 - acc: 0.5428
Epoch 10/20
98863/98863 [==============================] - 446s 5ms/sampl

In [0]:
model.save('ss.h5')

**Text Generation**

In [0]:
string_mapped=x[2]
full_string = [n_to_char[value] for value in string_mapped]
for i in range(200):
  x1=np.reshape(string_mapped,(1,len(string_mapped),1))
  x1=x1/float(len(characters))
  pred_index=np.argmax(model.predict(x1,verbose=0))
  
  full_string.append(n_to_char[pred_index])
  
  string_mapped.append(pred_index)
  string_mapped=string_mapped[1:len(string_mapped)]

In [89]:
txt=''
for char in full_string:
  txt=txt+char
txt

"from fairest creatures we desire increase,\n  that thereby beauty's rose might never die,\n  but as the world s due, by the spres so freat:\n  the canker vistue once more wilt to booter;\n  which bountelus says i did strive to prove\n  and him as fast as thou forget's fnes   with mine own shadow strposed"